In [35]:
import plotly.plotly as py
import plotly
plotly.tools.set_credentials_file(username='amcdonne', api_key='B3jR7xq5Lt9yK0WGDkPt')
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import xlwings
import pyperclip
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, Event, State
import visdcc

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

df = pd.read_csv('Agents_Loan_Amount.csv')

In [175]:
print('No Loan Amount Count: ', len(df[df.loan_amount.isin(['nan'])].index))

print('Row Count Original: ', len(df.index))
df = df.dropna().reset_index()
print('Row Count After NaN Drop: ', len(df.index))

cases = list(df['Unnamed: 0'])
#loan_amount = list(df['loan_amount'])
admin_agent = list(df['admin_agent'])
print('Total Case and Admin Observations: ', len(cases) + len(admin_agent))

edge_list = []
for i in range(0, len(df.index)):
    edge_list.append((str(df.loc[i]['admin_agent']), str(df.loc[i]['Unnamed: 0']), int(df.loc[i]['loan_amount'])))
print('Total Case Admin Pairs: ', len(edge_list))

No Loan Amount Count:  0
Row Count Original:  342
Row Count After NaN Drop:  342
Total Case and Admin Observations:  684
Total Case Admin Pairs:  342


In [176]:
G = nx.DiGraph()

G.add_nodes_from(cases)
G.add_nodes_from(admin_agent)

G.add_weighted_edges_from(edge_list, weight='weight')
#Sets the edge attributes: nx.get_edge_attributes(G, 'weight')

print('Number of Nodes: ', G.number_of_nodes())
print('Number of Edges: ', G.number_of_edges())

pos = nx.spring_layout(G)
#pos = nx.get_node_attributes(G,'pos')
#position of each node in the layout of the graph
print(isinstance(pos, dict))

nx.set_node_attributes(G, pos, 'pos')

print('First Node and Data: ', list(G.nodes.data())[0])
print('First Edge and Data: ', list(G.edges.data())[0])
#if the number of Case:Admin pairs differes from the # of edges, there are duplicates present
# Number of nodes differs from Case + Admin count -> there are duplicate entries

connections = {}
for n, nbrsdict in G.adjacency():
    #print(n, nbrsdict.keys())
    #print(n, len(nbrsdict.keys()))
    connections[n] = len(nbrsdict.keys())
print('First Node and Data: ', list(G.nodes.data())[0])
nx.set_node_attributes(G, connections, 'connections')

Number of Nodes:  454
Number of Edges:  312
True
First Node and Data:  ('In re Fairway Group Holdings Corp.', {'pos': array([0.95446769, 0.11245966])})
First Edge and Data:  ('Credit Suisse', 'In re Fairway Group Holdings Corp.', {'weight': 30611941})
First Node and Data:  ('In re Fairway Group Holdings Corp.', {'pos': array([0.95446769, 0.11245966])})


In [178]:
nx.get_node_attributes(G, 'connections')['Wells Fargo Bank']

27

{'pos': array([0.41232556, 0.58688956]), 'connections': 4}
{'pos': array([0.41232556, 0.58688956]), 'connections': 4}
{'pos': array([0.22234709, 0.02894776]), 'connections': 16}
{'pos': array([0.22234709, 0.02894776]), 'connections': 16}
{'pos': array([-0.14158199,  0.37674413]), 'connections': 7}
{'pos': array([-0.14158199,  0.37674413]), 'connections': 7}
{'pos': array([-0.14158199,  0.37674413]), 'connections': 7}
{'pos': array([0.58395284, 0.32096656]), 'connections': 1}
{'pos': array([0.06760469, 0.03351641]), 'connections': 16}
{'pos': array([0.55953508, 0.57900184]), 'connections': 1}
{'pos': array([0.1219184 , 0.85661305]), 'connections': 1}
{'pos': array([0.22234709, 0.02894776]), 'connections': 16}
{'pos': array([ 0.67255673, -0.4552604 ]), 'connections': 1}
{'pos': array([-0.04564563,  0.76400405]), 'connections': 1}
{'pos': array([-0.15120448, -0.0714742 ]), 'connections': 27}
{'pos': array([ 0.61954629, -0.80130167]), 'connections': 1}
{'pos': array([ 0.54598505, -0.702195

In [187]:
for n, nbrsdict in G.nodes.data():
    print(n, nbrsdict)

In re Fairway Group Holdings Corp. {'pos': array([0.95446769, 0.11245966]), 'connections': 0}
In re Arch Coal, Inc. {'pos': array([0.69679209, 0.6410857 ]), 'connections': 0}
In re Aspect Software Parent, Inc. {'pos': array([ 0.41897529, -0.90236888]), 'connections': 0}
In re Peabody Energy Corporation {'pos': array([ 0.9695304 , -0.31812941]), 'connections': 0}
In re AF-Southeast, LLC (Allied Fiber) {'pos': array([0.89604144, 0.4993884 ]), 'connections': 0}
In re CJ Holding Co. (C&amp;J Energy) {'pos': array([-0.87395457, -0.22185189]), 'connections': 0}
In re AOG Entertainment, Inc. (Core Entertainment) {'pos': array([0.6937781 , 0.72792441]), 'connections': 0}
In re Cosi, Inc. {'pos': array([0.127092  , 0.89352486]), 'connections': 0}
In re American Gilsonite Company {'pos': array([-0.24650732,  0.95742426]), 'connections': 0}
In re AstroTurf, LLC {'pos': array([ 0.8364556 , -0.56590568]), 'connections': 0}
In re Phoenix Brands LLC {'pos': array([-0.05475825,  0.93839346]), 'connect

In [202]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.3,color='#888'),
    hoverinfo='none',
    mode='lines')
#print(edge_trace['x'])
#print(type(edge_trace['x']))

# loops through edges, pulls out the case name and admin name; finds each names node position 
for edge in G.edges():
    #print(edge)
    x0, y0 = G.node[edge[0]]['pos']
    #print(G.node[edge[0]]['pos'])
    x1, y1 = G.node[edge[1]]['pos']
    #print(G.node[edge[1]]['pos'])
    
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

In [207]:
node_x = []
node_y = []
color_list = []
text_list = []
size_list = []

for case in cases:
    #print(G.node[node]['pos'])
    G.nodes.data()[case]
    node_x.append(G.node[case]['pos'][0])
    node_y.append(G.node[case]['pos'][1])
    #text_list.append([G.node[node]['pos'][0], G.node[node]['pos'][1], node])
    connect = G.nodes.data()[case]['connections']
    color_list.append(connect)
    text_list.append([str(case), 'Connections: '+str(connect)])

case_trace = go.Scatter(
    x=node_x,
    y=node_y,
    text=text_list,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=color_list,
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

In [208]:
node_x = []
node_y = []
color_list = []
text_list = []
size_list = []

for agent in admin_agent:
    #print(G.node[node]['pos'])
    G.nodes.data()[agent]
    node_x.append(G.node[agent]['pos'][0])
    node_y.append(G.node[agent]['pos'][1])
    #text_list.append([G.node[node]['pos'][0], G.node[node]['pos'][1], node])
    connect = G.nodes.data()[agent]['connections']
    color_list.append(connect)
    text_list.append([str(agent), 'Connections: '+str(connect)])

admin_trace = go.Scatter(
    x=node_x,
    y=node_y,
    text=text_list,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=color_list,
        size=10,
        symbol='square',
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

In [210]:
fig = go.Figure(data=[case_trace, admin_trace])
py.iplot(fig, filename='nodes')

In [211]:
fig = go.Figure(data=[edge_trace, case_trace, admin_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text='',
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='network')